In [5]:
#load gamelog_data
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
import json
import requests
import datetime

def season_subset(df, year_season_start, year_season_end = None):
    df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])
    if year_season_end is None:
        year_season_end = year_season_start + 1
    df_gt = df[df.GAME_DATE > datetime.date(year_season_start,9,1)]
    df_lt = df_gt[df_gt.GAME_DATE < datetime.date(year_season_end,9,1)]
    return df_lt.sort_values("GAME_DATE") if not df_lt.empty else None

In [8]:
post85df = pd.read_csv('./gamelogs/master_post86df.csv')
post85df = post85df.drop('VIDEO_AVAILABLE',1)
df85_15 = season_subset(post85df,1985,2015)

by_player = df85_15.groupby("PLAYER_NAME")

In [9]:
df85_15["FANTASY_ZSCORE"] = by_player["FANTASY_PTS"].apply(lambda x: ((x - x.mean())/x.std()))
df85_15["i_ZSCORE_OVER"] = df85_15["FANTASY_ZSCORE"].map(lambda x: 1 if x > 1 else 0)
df85_15["SEASON_MIN"] = by_player['MIN'].apply(lambda x: x.map(lambda y: x.sum()))
df85_15["GAMES_PLAYED"] = by_player["PLAYER_NAME"].apply(lambda x: x.map(lambda y: len(x)))
for x in ['FG_PCT', 'FG3_PCT', 'FT_PCT']:
    df85_15[x] = df85_15[x].map(lambda y: 0 if np.isnan(y) else y)
df85_15["WL"] = [1 if v == "W" else 0 for v in df85_15.copy()["WL"]]

opp_home = df85_15.MATCHUP.map(lambda x: (x[-3:],0) if "@" in x else (x[-3:],1))
df85_15["OPP"] = opp_home.map(lambda x: x[0])
df85_15["i_HOME"] = opp_home.map(lambda x: x[1])

In [10]:
#Add player bio data for age,weight,height
player_bios_df = pd.read_csv("./player_bios/player_bios.csv")
player_bios_df = player_bios_df.rename(columns = {'PERSON_ID': 'PLAYER_ID', 'DISPLAY_FIRST_LAST': 'PLAYER_NAME'})
player_bios_df["BIRTHDATE"] = pd.to_datetime(player_bios_df["BIRTHDATE"])
player_bios_df['AGE'] = player_bios_df["BIRTHDATE"].map(lambda x: round((pd.to_datetime('today') - x).days / 365.,2))
player_bios_df["WEIGHT"] = player_bios_df["WEIGHT"].astype('str')
player_bios_df["HEIGHT"] = player_bios_df["HEIGHT"].astype('str')
player_bios_df["WEIGHT"] = player_bios_df["WEIGHT"].map(lambda x:  float(x) if x != 'nan' else 0.)
player_bios_df["HEIGHT"] = player_bios_df["HEIGHT"].map(lambda x: (12.*float(x[0]) + float(x[2:])) if x != 'nan' else 0.)

by_player = df85_15.groupby("PLAYER_NAME")

In [11]:
def get_player_bio(name, col_name):
    return float(player_bios_df[player_bios_df.PLAYER_NAME == name][col_name])

df85_15["AGE"] = by_player["PLAYER_NAME"].apply(lambda x: x.replace(x.iloc[0],get_player_bio(x.iloc[0],"AGE")))
df85_15["WEIGHT"] = by_player["PLAYER_NAME"].apply(lambda x: x.replace(x.iloc[0],get_player_bio(x.iloc[0],"WEIGHT")))
df85_15["HEIGHT"] = by_player["PLAYER_NAME"].apply(lambda x: x.replace(x.iloc[0],get_player_bio(x.iloc[0],"HEIGHT")))

In [12]:
#Integrate ELO Rankings
elo_df = pd.read_csv("./gamelogs/all_elo.csv")
elo_df["date_game"] = pd.to_datetime(elo_df["date_game"])
elo_df["game_location"] = elo_df["game_location"].map(lambda x: 1 if x == "H" else 0)
elo_df = elo_df[elo_df["is_playoffs"] == 0]

curr = elo_df.columns.tolist()
cols = [curr[i] for i in [5,8,11,13,14,17,19,21]]
elo_df = elo_df[cols]
elo_df = elo_df.rename(columns={'date_game': 'GAME_DATE',
                                'team_id':'TEAM_ABBREVIATION',
                                'opp_id':'OPP', 
                                'game_location': 'i_HOME',
                                'elo_i':'ELO',
                                'opp_elo_i': 'OPP_ELO',
                                'win_equiv': 'EXP_WINS',
                                'forecast':'FORECAST'})

df85_15 = df85_15.merge(season_subset(elo_df,1985,2015))

In [13]:
#Rearrange some columns in df85_15
curr = df85_15.columns.tolist()
cols = curr[:3] + curr[32:37] + curr[3:9] + curr[37:] + curr[9:32]
if len(curr) == len(cols):
    df85_15 = df85_15[cols]


name_pos = player_bios_df[["PLAYER_ID","POSITION","PLAYER_NAME"]]
df85_15 = df85_15.merge(name_pos)
df85_15.columns.tolist()

['SEASON_ID',
 'PLAYER_ID',
 'PLAYER_NAME',
 'SEASON_MIN',
 'GAMES_PLAYED',
 'OPP',
 'i_HOME',
 'AGE',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'WEIGHT',
 'HEIGHT',
 'ELO',
 'EXP_WINS',
 'OPP_ELO',
 'FORECAST',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'PLUS_MINUS',
 'FANTASY_PTS',
 'FANTASY_ZSCORE',
 'i_ZSCORE_OVER',
 'POSITION']

In [14]:
def calc_season_avg(df,col_list,(date_str1,date_str2)):
    date1, date2 = pd.to_datetime(date_str1), pd.to_datetime(date_str2)
    mask = lambda x: (date1 <= x) & (x <= date2)
    return df[df.GAME_DATE.apply(mask)].groupby(["PLAYER_NAME","SEASON_ID"])[col_list].mean().reset_index()

In [15]:
def ngames_colname(col_list, ngames):
    return map(lambda x: str(ngames) + 'D_' + x, col_list)

In [16]:
def last_ngames(df,ngames,game_date,col_list):
    ngames_df = df[df.GAME_DATE < game_date].nlargest(ngames, "GAME_DATE")
    ngames_col_list = ngames_colname(col_list,ngames)
    num_cols = len(ngames_col_list)
    date_player_tuples = [("GAME_DATE",game_date)]#,("PLAYER_NAME",df.PLAYER_NAME.iloc[0])]
    if ngames_df.empty:
        return dict(date_player_tuples + zip(ngames_col_list,np.array(0).repeat(num_cols)))
    else:
        return dict(date_player_tuples + zip(ngames_col_list,ngames_df[col_list].mean()))

In [17]:
def calc_ngame_avg(df,col_list,game_date_str,ngames):
    game_date = pd.to_datetime(game_date_str)
    season_id = df[df.GAME_DATE == game_date]["SEASON_ID"].iloc[0]
    return last_ngames(df[df.SEASON_ID == season_id],ngames,game_date,col_list)

In [107]:
def rolling_cols(df,col_list,ngames,rolling_kind):
    if rolling_kind == 'mean':
        rolling_func = lambda (a,b,c): pd.rolling_mean(a,b,min_periods = c)
    elif rolling_kind == 'sum':
        rolling_func = lambda (a,b,c): pd.rolling_sum(a,b,min_periods = c)
    else:
        return None 
    
    rolling_df = (df.groupby(["PLAYER_NAME","SEASON_ID", "OPP"])
                    .apply(lambda x: add_game_date_pts_col(rolling_func((x[col_list],ngames,1)),x.GAME_DATE).reset_index(drop = True)))
    return rolling_df.reset_index().rename(columns=dict(zip(col_list,map(lambda x: 'R_' + x,col_list))))

In [103]:
def add_game_date_pts_col(df,game_date_col):
   new_df = pd.concat([df,game_date_col], axis = 1)
   return new_df

In [20]:
def per_season_cumsum(df,col_list):
    cumsum_df = (df.groupby(["PLAYER_NAME","SEASON_ID"])
                   .apply(lambda x: add_game_date_col(x[col_list].cumsum(axis = 0), x.GAME_DATE).reset_index(drop = True)))
    return cumsum_df.reset_index().drop('level_2',axis = 1).rename(columns=dict(zip(col_list,map(lambda x: 'C_' + x,col_list))))

In [21]:
def per_season_cummean(df,col_list):
    cumsum_df = (df.groupby(["PLAYER_NAME","SEASON_ID"])
                   .apply(lambda x: add_game_date_pts_col(pd.expanding_mean(x[col_list], min_periods = 2), x.GAME_DATE, x.FANTASY_PTS).reset_index(drop = True)))
    return cumsum_df.reset_index().drop('level_2',axis = 1).rename(columns=dict(zip(col_list,map(lambda x: 'C_' + x,col_list))))

In [22]:
def get_player_seasons(player_name, season1,season2,full_df):
    player_df = (full_df[full_df.PLAYER_NAME == player_name].groupby(["PLAYER_NAME","SEASON_ID"])
                                .apply(lambda x: pd.DataFrame(map(lambda y: calc_ngame_avg(x.sort_values("GAME_DATE"),["AST","REB","PTS","TOV","STL","BLK"],y,3),x.GAME_DATE)))).reset_index().drop('level_2',axis = 1)
    player_df = pd.merge(player_df,full_df[full_df.PLAYER_NAME == player_name][["GAME_DATE","FANTASY_PTS","OPP_ELO"]])
    player_df['SHIT'] = player_df['OPP_ELO'].map(lambda x: 1 if x < 1400 else 0)
    player_df['OKAY'] = player_df['OPP_ELO'].map(lambda x: 1 if 1400 <= x < 1600 else 0)
    player_df['GOOD'] = player_df['OPP_ELO'].map(lambda x: 1 if 1600 <= x < 1700 else 0)
    player_df['GREAT'] = player_df['OPP_ELO'].map(lambda x: 1 if 1700 <= x else 0)
    player_df2 = player_df.set_index('GAME_DATE')
    fantasy_resp = player_df2.groupby('SEASON_ID').apply(lambda x: x['FANTASY_PTS'].map(lambda y: 1 if y > x.FANTASY_PTS.mean() else 0)).reset_index().rename(columns={'FANTASY_PTS':'FANTASY_RESP'})
    player_df2 = pd.merge(player_df2,fantasy_resp)
    fst_season = season1 + 20000
    lst_season = season2 + 20000
    player_df_final = player_df2[(player_df2.SEASON_ID <= lst_season) & (player_df2.SEASON_ID >= fst_season)].sort_values('SEASON_ID')
    return (player_df_final, np.array(player_df_final.SEASON_ID < lst_season))

In [23]:
df,mask = get_player_seasons("Roy Hibbert",2010,2013,df85_15) 

In [26]:
%connect_info

{
  "stdin_port": 60990, 
  "ip": "127.0.0.1", 
  "control_port": 60991, 
  "hb_port": 60992, 
  "signature_scheme": "hmac-sha256", 
  "key": "ecc4c8e9-a45a-419c-8f79-6748a03165d8", 
  "shell_port": 60988, 
  "transport": "tcp", 
  "iopub_port": 60989
}

Paste the above JSON into a file, and connect with:
    $> ipython <app> --existing <file>
or, if you are local, you can connect with just:
    $> ipython <app> --existing kernel-0876d50e-7e06-4cbb-bd0c-f4ebf1e5cdbf.json 
or even just:
    $> ipython <app> --existing 
if this is the most recent IPython session you have started.


In [ ]:
df.head()
#train_test_split(xrange(df.shape[0]), train_size=0.7)
mask.shape,mask.sum()

dftouse = df.copy()

STANDARDIZABLE = map(lambda x: '3D_' + x,["AST","REB","PTS","TOV","STL","BLK"])
from sklearn.preprocessing import StandardScaler
for col in STANDARDIZABLE:
    print col
    valstrain=df[col].values[mask]
    valstest=df[col].values[~mask]
    scaler=StandardScaler().fit(valstrain)
    outtrain=scaler.transform(valstrain)
    outtest=scaler.fit_transform(valstest)
    out=np.empty(mask.shape[0])
    out[mask]=outtrain
    out[~mask]=outtest
    dftouse[col]=out

lcols = STANDARDIZABLE + ["SHIT","OKAY","GOOD","GREAT"]

from sklearn import svm 
clfsvm = svm.SVC(kernel = 'rbf')
cs=[.01,.1,1,10,100]
gammas = range(0,101,5)
Xmatrix=dftouse[lcols].values
Yresp=dftouse['FANTASY_RESP'].values 
Xmatrix_train=Xmatrix[mask]
Xmatrix_test=Xmatrix[~mask]
Yresp_train=Yresp[mask]
Yresp_test=Yresp[~mask]
df[~mask].tail()

#your code here
from sklearn.grid_search import GridSearchCV
gs=GridSearchCV(clfsvm, param_grid={'C':cs,'gamma':gammas}, cv=5)
gs.fit(Xmatrix_train, Yresp_train)
print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_

#calculate the accuracy here, bullshit
best = gs.best_estimator_
best.fit(Xmatrix_train, Yresp_train)
best.score(Xmatrix_test, Yresp_test)

In [27]:
df

,PLAYER_NAME,SEASON_ID,3D_AST,3D_BLK,3D_PTS,3D_REB,3D_STL,3D_TOV,FANTASY_PTS,OPP_ELO,SHIT,OKAY,GOOD,GREAT,GAME_DATE,FANTASY_RESP
10865,Roy Hibbert,22010,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,48.25,1594.2832,0,1,0,0,2010-10-27,1
15032,Roy Hibbert,22010,4.000000,1.666667,20.000000,8.000000,0.000000,2.666667,32.50,1347.6910,1,0,0,0,2011-03-08,0
15031,Roy Hibbert,22010,4.000000,1.666667,20.000000,8.000000,0.000000,2.666667,32.50,1347.6910,1,0,0,0,2011-03-05,1
15030,Roy Hibbert,22010,4.000000,1.666667,20.000000,8.000000,0.000000,2.666667,32.50,1347.6910,1,0,0,0,2011-03-04,0
15029,Roy Hibbert,22010,4.000000,1.666667,20.000000,8.000000,0.000000,2.666667,32.50,1347.6910,1,0,0,0,2011-03-02,1
15028,Roy Hibbert,22010,4.000000,1.666667,20.000000,8.000000,0.000000,2.666667,32.50,1347.6910,1,0,0,0,2011-03-01,1
15027,Roy Hibbert,22010,4.000000,1.666667,20.000000,8.000000,0.000000,2.666667,32.50,1347.6910,1,0,0,0,2011-02-25,1
15026,Roy Hibbert,22010,4.000000,1.666667,20.000000,8.000000,0.000000,2.666667,32.50,1347.6910,1,0,0,0,2011-02-23,0
15025,Roy Hibbert,22010,4.000000,1.666667,20.000000,8.000000,0.000000,2.666667,32.50,1347.6910,1,0,0,0,2011-02-22,1
15024,Roy Hibbert,22010,4.000000,1.666667,20.000000,8.000000,0.000000,2.666667,32.50,1347.6910,1,0,0,0,2011-02-16,1


In [28]:
from scipy 

SyntaxError: invalid syntax (<ipython-input-28-f05e4a76f6f8>, line 1)

In [29]:
test = per_season_cummean(df85_15,['FANTASY_PTS'])

In [30]:
pd.expanding_mean(pd.Series([1,2,3,4,5,6,7]), min_periods = 3)

0    NaN
1    NaN
2    2.0
3    2.5
4    3.0
5    3.5
6    4.0
dtype: float64

In [31]:
green = df85_15[df85_15.PLAYER_NAME == "A.C. Green"]
add_game_date_pts_col(pd.expanding_mean(df85_15[df85_15.PLAYER_NAME == "A.C. Green"]['FANTASY_PTS']), green.GAME_DATE, green.FANTASY_PTS)

,FANTASY_PTS,GAME_DATE
45841,42.500000,1985-10-29
45842,27.625000,1985-11-05
45843,26.500000,1985-11-08
45844,21.437500,1985-11-14
45845,23.950000,1985-11-15
45846,23.833333,1985-11-17
45847,23.142857,1985-11-20
45848,22.593750,1985-11-21
45849,22.027778,1985-11-23
45850,20.325000,1985-11-29


In [32]:
;

''

In [33]:
test[test.PLAYER_NAME == 'A.C. Green'][test.SEASON_ID == 21985]

/Applications/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


,PLAYER_NAME,SEASON_ID,C_FANTASY_PTS,GAME_DATE
0,A.C. Green,21985,NaN,1985-10-29
1,A.C. Green,21985,27.625000,1985-11-05
2,A.C. Green,21985,26.500000,1985-11-08
3,A.C. Green,21985,21.437500,1985-11-14
4,A.C. Green,21985,23.950000,1985-11-15
5,A.C. Green,21985,23.833333,1985-11-17
6,A.C. Green,21985,23.142857,1985-11-20
7,A.C. Green,21985,22.593750,1985-11-21
8,A.C. Green,21985,22.027778,1985-11-23
9,A.C. Green,21985,20.325000,1985-11-29


In [35]:
cumsum_df.reset_index().drop('level_2', axis = 1).rename(columns=dict(zip(["FANTASY_PTS"],map(lambda x: 'C_' + x,["FANTASY_PTS"]))))

NameError: name 'cumsum_df' is not defined

In [60]:
df85_15.groupby(["PLAYER_NAME", 'OPP']).apply(lambda x: x.FANTASY_PTS).reset_index().drop('level_2', axis =1)

,PLAYER_NAME,OPP,FANTASY_PTS
0,A.C. Green,ATL,10.00
1,A.C. Green,ATL,11.25
2,A.C. Green,ATL,30.25
3,A.C. Green,ATL,27.75
4,A.C. Green,ATL,25.50
5,A.C. Green,ATL,31.25
6,A.C. Green,ATL,36.75
7,A.C. Green,ATL,35.50
8,A.C. Green,ATL,33.25
9,A.C. Green,ATL,18.00


In [86]:
df85_15

,SEASON_ID,PLAYER_ID,PLAYER_NAME,SEASON_MIN,GAMES_PLAYED,OPP,i_HOME,AGE,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,WEIGHT,HEIGHT,ELO,EXP_WINS,OPP_ELO,FORECAST,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FANTASY_PTS,FANTASY_ZSCORE,i_ZSCORE_OVER,POSITION
0,21985,77377,Cliff Levingston,12289,613,MIL,0,54.95,ATL,Atlanta Hawks,28500013,1985-10-26,ATL @ MIL,0,210,80,1464.5355,36.319843,1604.5205,0.200774,30,1,3,0.333,0,0,0.000,0,0,0.000,3,6,9,0,2,1,1,6,2,NaN,18.75,0.233651,0,Forward
1,21985,77377,Cliff Levingston,12289,613,NYK,1,54.95,ATL,Atlanta Hawks,28500030,1985-10-29,ATL vs. NYK,1,210,80,1457.8932,36.926842,1367.5015,0.749507,32,0,3,0.000,0,0,0.000,5,8,0.625,5,5,10,3,0,1,1,1,5,NaN,23.50,0.684053,0,Forward
2,21985,77377,Cliff Levingston,12289,613,BOS,0,54.95,ATL,Atlanta Hawks,28500045,1985-11-01,ATL @ BOS,0,210,80,1463.7467,36.750767,1646.1967,0.164391,22,4,6,0.667,0,0,0.000,3,4,0.750,2,2,4,1,0,0,0,4,11,NaN,17.50,0.115124,0,Forward
3,21985,77377,Cliff Levingston,12289,613,DEN,0,54.95,ATL,Atlanta Hawks,28500060,1985-11-05,ATL @ DEN,0,210,80,1466.6379,36.760952,1608.5680,0.198983,22,5,6,0.833,0,0,0.000,2,2,1.000,2,2,4,1,0,0,2,3,12,NaN,17.50,0.115124,0,Forward
4,21985,77377,Cliff Levingston,12289,613,LAC,0,54.95,ATL,Atlanta Hawks,28500085,1985-11-09,ATL @ LAC,1,210,80,1461.4900,37.467758,1447.4091,0.378813,28,9,14,0.643,0,0,0.000,2,3,0.667,5,6,11,1,2,1,3,4,20,NaN,39.75,2.224904,1,Forward
5,21985,77377,Cliff Levingston,12289,613,DET,1,54.95,ATL,Atlanta Hawks,28500117,1985-11-15,ATL vs. DET,1,210,80,1472.4945,38.467503,1560.0360,0.517922,34,8,13,0.615,0,0,0.000,5,8,0.625,3,7,10,1,1,2,3,1,21,NaN,39.50,2.201198,1,Forward
6,21985,77377,Cliff Levingston,12289,613,NYK,0,54.95,ATL,Atlanta Hawks,28500119,1985-11-16,ATL @ NYK,0,210,80,1478.5320,37.480686,1345.5177,0.547369,37,7,12,0.583,0,0,0.000,3,5,0.600,1,3,4,2,1,2,1,3,17,NaN,30.50,1.347804,1,Forward
7,21985,77377,Cliff Levingston,12289,613,CHI,1,54.95,ATL,Atlanta Hawks,28500141,1985-11-20,ATL vs. CHI,1,210,80,1469.0724,38.252716,1426.9819,0.693799,36,6,16,0.375,0,0,0.000,1,2,0.500,7,10,17,3,2,1,2,3,13,NaN,43.75,2.604190,1,Forward
8,21985,77377,Cliff Levingston,12289,613,CLE,0,54.95,ATL,Atlanta Hawks,28500166,1985-11-24,ATL @ CLE,0,210,80,1463.7317,36.418217,1510.2817,0.300775,36,6,13,0.462,0,0,0.000,5,8,0.625,5,6,11,3,3,1,4,4,17,NaN,41.25,2.367136,1,Forward
9,21985,77377,Cliff Levingston,12289,613,NYK,1,54.95,ATL,Atlanta Hawks,28500175,1985-11-26,ATL vs. NYK,1,210,80,1458.8431,36.849419,1352.2932,0.766562,24,5,7,0.714,0,0,0.000,2,2,1.000,3,4,7,1,0,0,1,6,12,NaN,21.75,0.518116,0,Forward


In [92]:
merged_df = pd.merge(df85_15,df85_15.groupby(["PLAYER_NAME",'OPP']).apply(lambda x: x.FANTASY_PTS).reset_index())#.rename(columns = {0:"FANTASY_PTS_AGAINST"}), left_on=['PLAYER_NAME','OPP'],right_on=['PLAYER_NAME','OPP'])

In [94]:
testdata = season_subset(merged_df, 2005, 2006)

In [111]:
high_rollers = testdata.groupby(["PLAYER_NAME",'OPP']).apply(lambda x: rolling_cols(x,['FANTASY_PTS'],3,'mean'))

In [123]:
high_rollers

PLAYER_NAME  SEASON_ID  OPP  level_3  R_FANTASY_PTS  GAME_DATE
PLAYER_NAME        OPP                                                                         
Aaron McKie        ATL 0         Aaron McKie      22005  ATL        0       0.000000 2005-11-08
                       1         Aaron McKie      22005  ATL        1       0.000000 2005-11-08
                   CHI 0         Aaron McKie      22005  CHI        0       5.000000 2005-11-20
                   DEN 0         Aaron McKie      22005  DEN        0       3.750000 2005-11-02
                       1         Aaron McKie      22005  DEN        1       5.000000 2005-11-06
                   HOU 0         Aaron McKie      22005  HOU        0       1.250000 2006-04-02
                   LAC 0         Aaron McKie      22005  LAC        0       5.000000 2005-11-18
                   NOK 0         Aaron McKie      22005  NOK        0      11.500000 2006-04-19
                   PHI 0         Aaron McKie      22005  PHI        0       2.750000 2005-11-11
                   POR 0         Aaron McKie      22005  POR        0       0.000000 2006-04-14
                   SAS 0         Aaron McKie      22005  SAS        0       8.500000 2005-11-29
                   SEA 0         Aaron McKie      22005  SEA        0       1.250000 2005-11-24
                   UTA 0         Aaron McKie      22005  UTA        0       3.500000 2005-12-01
Aaron Miles        ATL 0         Aaron Miles      22005  ATL        0       9.000000 2005-11-02
                   BOS 0         Aaron Miles      22005  BOS        0       2.000000 2005-12-19
                       1         Aaron Miles      22005  BOS        1       1.500000 2005-12-28
                   CHA 0         Aaron Miles      22005  CHA        0       0.000000 2005-12-02
                   CHI 0         Aaron Miles      22005  CHI        0       1.250000 2005-11-14
                   LAC 0         Aaron Miles      22005  LAC        0       2.500000 2005-11-20
                   NJN 0         Aaron Miles      22005  NJN        0       4.250000 2005-11-21
                       1         Aaron Miles      22005  NJN        1       4.625000 2005-12-18
                   NOK 0         Aaron Miles      22005  NOK        0       8.750000 2005-11-28
                   NYK 0         Aaron Miles      22005  NYK        0       0.000000 2005-11-06
                       1         Aaron Miles      22005  NYK        1       0.000000 2005-11-06
                       2         Aaron Miles      22005  NYK        2       0.000000 2005-11-11
                       3         Aaron Miles      22005  NYK        3       0.000000 2005-11-11
                   POR 0         Aaron Miles      22005  POR        0       0.000000 2005-11-18
                   SAC 0         Aaron Miles      22005  SAC        0       1.500000 2005-11-30
                   SAS 0         Aaron Miles      22005  SAS        0       8.250000 2005-11-23
                   TOR 0         Aaron Miles      22005  TOR        0       6.750000 2005-11-26
...                                      ...        ...  ...      ...            ...        ...
Zydrunas Ilgauskas NYK 1  Zydrunas Ilgauskas      22005  NYK        1      34.625000 2006-04-13
                   ORL 0  Zydrunas Ilgauskas      22005  ORL        0      26.750000 2005-11-13
                       1  Zydrunas Ilgauskas      22005  ORL        1      35.250000 2005-11-18
                       2  Zydrunas Ilgauskas      22005  ORL        2      37.583333 2006-02-21
                       3  Zydrunas Ilgauskas      22005  ORL        3      42.750000 2006-02-21
                       4  Zydrunas Ilgauskas      22005  ORL        4      34.083333 2006-03-10
                       5  Zydrunas Ilgauskas      22005  ORL        5      25.916667 2006-03-10
                   PHI 0  Zydrunas Ilgauskas      22005  PHI        0      19.250000 2005-11-19
                       1  Zydrunas Ilgauskas      22005  PHI        1      32.500000 2006-02-04
 

In [125]:
high_rollers[high_rollers.PLAYER_NAME == 'Kobe Bryant']

PLAYER_NAME  SEASON_ID  OPP  level_3  R_FANTASY_PTS  GAME_DATE
PLAYER_NAME OPP                                                                  
Kobe Bryant ATL 0  Kobe Bryant      22005  ATL        0      51.750000 2005-11-08
                1  Kobe Bryant      22005  ATL        1      56.250000 2006-02-15
            BOS 0  Kobe Bryant      22005  BOS        0      61.000000 2006-02-26
                1  Kobe Bryant      22005  BOS        1      62.625000 2006-03-20
            CHA 0  Kobe Bryant      22005  CHA        0      43.500000 2005-12-04
                1  Kobe Bryant      22005  CHA        1      43.500000 2005-12-04
                2  Kobe Bryant      22005  CHA        2      48.750000 2006-02-03
            CHI 0  Kobe Bryant      22005  CHI        0      58.000000 2005-11-20
                1  Kobe Bryant      22005  CHI        1      52.125000 2005-12-09
            CLE 0  Kobe Bryant      22005  CLE        0      34.500000 2006-01-12
                1  Kobe Bryant      22005  CLE        1      47.250000 2006-03-19
            DAL 0  Kobe Bryant      22005  DAL        0      62.750000 2005-12-12
                1  Kobe Bryant      22005  DAL        1      70.875000 2005-12-20
                2  Kobe Bryant      22005  DAL        2      60.750000 2006-02-07
            DEN 0  Kobe Bryant      22005  DEN        0      48.250000 2005-11-02
                1  Kobe Bryant      22005  DEN        1      51.375000 2005-11-06
                2  Kobe Bryant      22005  DEN        2      52.416667 2005-11-06
                3  Kobe Bryant      22005  DEN        3      54.166667 2006-04-06
            DET 0  Kobe Bryant      22005  DET        0      53.000000 2006-01-29
                1  Kobe Bryant      22005  DET        1      52.375000 2006-03-04
            GSW 0  Kobe Bryant      22005  GSW        0      56.250000 2006-01-14
                1  Kobe Bryant      22005  GSW        1      51.250000 2006-01-27
                2  Kobe Bryant      22005  GSW        2      54.000000 2006-03-03
                3  Kobe Bryant      22005  GSW        3      51.250000 2006-04-11
            HOU 0  Kobe Bryant      22005  HOU        0      38.500000 2005-12-18
                1  Kobe Bryant      22005  HOU        1      51.375000 2006-02-08
                2  Kobe Bryant      22005  HOU        2      53.666667 2006-04-02
            IND 0  Kobe Bryant      22005  IND        0      67.000000 2006-01-09
                1  Kobe Bryant      22005  IND        1      56.375000 2006-02-01
                2  Kobe Bryant      22005  IND        2      52.833333 2006-02-01
...                        ...        ...  ...      ...            ...        ...
            PHI 0  Kobe Bryant      22005  PHI        0      39.250000 2005-11-11
                1  Kobe Bryant      22005  PHI        1      53.375000 2006-01-06
            POR 0  Kobe Bryant      22005  POR        0      56.500000 2006-01-11
                1  Kobe Bryant      22005  POR        1      55.250000 2006-02-21
                2  Kobe Bryant      22005  POR        2      54.166667 2006-03-01
                3  Kobe Bryant      22005  POR        3      58.666667 2006-04-14
            SAC 0  Kobe Bryant      22005  SAC        0      70.250000 2006-01-19
                1  Kobe Bryant      22005  SAC        1      67.125000 2006-02-23
                2  Kobe Bryant      22005  SAC        2      66.083333 2006-02-23
                3  Kobe Bryant      22005  SAC        3      60.583333 2006-03-14
                4  Kobe Bryant      22005  SAC        4      54.333333 2006-03-22
            SAS 0  Kobe Bryant      22005  SAS        0      38.500000 2005-11-29
                1  Kobe Bryant      22005  SAS        1      38.500000 2005-11-29
                2  Kobe Bryant      22005  SAS        2      44.166667 2006-03-06
                3  Kobe Bryant      22005  SAS        3      45.833333 2006-03-10
                4  Kobe Bryant      22005  SAS        4      47.500000 2006-03-10
 

In [119]:
kobe_hold

PLAYER_NAME         OPP   
Aaron McKie         ATL  0    False
                         1    False
                    CHI  0    False
                    DEN  0    False
                         1    False
                    HOU  0    False
                    LAC  0    False
                    NOK  0    False
                    PHI  0    False
                    POR  0    False
                    SAS  0    False
                    SEA  0    False
                    UTA  0    False
Aaron Miles         ATL  0    False
                    BOS  0    False
                         1    False
                    CHA  0    False
                    CHI  0    False
                    LAC  0    False
                    NJN  0    False
                         1    False
                    NOK  0    False
                    NYK  0    False
                         1    False
                         2    False
                         3    False
                    POR  0    False
 